# Laboratorio de prácticas: Árboles de decisión

En este ejercicio, implementarás un árbol de decisión desde cero y lo aplicarás a la tarea de clasificar si una seta es comestible o venenosa.

# Outline
- [ 1 - Packages ](#1)
- [ 2 -  Problem Statement](#2)
- [ 3 - Dataset](#3)
  - [ 3.1 One hot encoded dataset](#3.1)
- [ 4 - Decision Tree Refresher](#4)
  - [ 4.1  Calculate entropy](#4.1)
    - [ Exercise 1](#ex01)
  - [ 4.2  Split dataset](#4.2)
    - [ Exercise 2](#ex02)
  - [ 4.3  Calculate information gain](#4.3)
    - [ Exercise 3](#ex03)
  - [ 4.4  Get best split](#4.4)
    - [ Exercise 4](#ex04)
- [ 5 - Building the tree](#5)


<a name="1"></a>
## 1 Paquetes 

En primer lugar, vamos a ejecutar la celda de abajo para importar todos los paquetes que necesitarás durante esta tarea.
- [numpy](https://www.numpy.org) es el paquete fundamental para trabajar con matrices en Python.
- [matplotlib](https://matplotlib.org) es una famosa biblioteca para trazar gráficos en Python.
- ``utils.py`` contiene funciones de ayuda para esta tarea. No es necesario modificar el código de este archivo.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from public_tests import *

%matplotlib inline

<a name="2"></a>
## 2 -  Planteamiento del problema

Supongamos que está creando una empresa que cultiva y vende setas silvestres. 
- Dado que no todas las setas son comestibles, le gustaría poder decir si una seta determinada es comestible o venenosa basándose en sus atributos físicos
- Tiene algunos datos existentes que puede utilizar para esta tarea. 

¿Puedes utilizar los datos para ayudarte a identificar qué setas se pueden vender de forma segura? 

Nota: El conjunto de datos utilizado es sólo para fines ilustrativos. No pretende ser una guía para identificar las setas comestibles.



<a name="3"></a>
## 3 - Conjunto de datos

Comenzará cargando el conjunto de datos para esta tarea. El conjunto de datos que ha recogido es el siguiente:


| Cap Color | Stalk Shape | Solitary | Edible |
|:---------:|:-----------:|:--------:|:------:|
|   Brown   |   Tapering  |    Yes   |    1   |
|   Brown   |  Enlarging  |    Yes   |    1   |
|   Brown   |  Enlarging  |    No    |    0   |
|   Brown   |  Enlarging  |    No    |    0   |
|   Brown   |   Tapering  |    Yes   |    1   |
|    Red    |   Tapering  |    Yes   |    0   |
|    Red    |  Enlarging  |    No    |    0   |
|   Brown   |  Enlarging  |    Yes   |    1   |
|    Red    |   Tapering  |    No    |    1   |
|   Brown   |  Enlarging  |    No    |    0   |


- Tienes 10 ejemplos de setas. Para cada ejemplo, tienes
    - Tres características
        - Color del sombrero (`Marrón` o `Rojo`),
        - Forma del tallo (`Tapering` o `Enlarging`), y
        - Solitario (`Sí` o `No`)
    - Etiqueta
        - Comestible (`1` que indica que sí o `0` que indica que es venenoso)

<a name="3.1"></a>
### 3.1 Conjunto de datos codificados en caliente (One hot encoded)
Para facilitar la aplicación, hemos codificado las características en caliente (convirtiéndolas en características de valor 0 o 1)

| Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:---------:|:--------------------:|:--------:|:------:|
|     1     |           1          |     1    |    1   |
|     1     |           0          |     1    |    1   |
|     1     |           0          |     0    |    0   |
|     1     |           0          |     0    |    0   |
|     1     |           1          |     1    |    1   |
|     0     |           1          |     1    |    0   |
|     0     |           0          |     0    |    0   |
|     1     |           0          |     1    |    1   |
|     0     |           1          |     0    |    1   |
|     1     |           0          |     0    |    0   |

Por lo tanto,
- `X_train` contiene tres características para cada ejemplo 
    - Color marrón (Un valor de `1` indica el color del sombrero "marrón" y `0` indica el color del sombrero "rojo")
    - Forma cónica (Un valor de `1` indica "forma cónica del tallo" y `0` indica forma de tallo "creciente")
    - Solitario (Un valor de `1` indica "Sí" y `0` indica "No")

- El valor de `y_train` indica si la seta es comestible 
    - `y = 1` indica que es comestible
    - y = 0` indica que es venenosa

In [4]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

#### Ver las variables
Vamos a familiarizarnos con el conjunto de datos.  
- Un buen punto de partida es imprimir cada variable y ver lo que contiene.

El código siguiente imprime los primeros elementos de `X_train` y el tipo de la variable.

In [5]:
print("First few elements of X_train:\n", X_train[:5])
print("Type of X_train:",type(X_train))

First few elements of X_train:
 [[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]]
Type of X_train: <class 'numpy.ndarray'>


Now, let's do the same for `y_train`

In [6]:
print("First few elements of y_train:", y_train[:5])
print("Type of y_train:",type(y_train))

First few elements of y_train: [1 1 0 0 1]
Type of y_train: <class 'numpy.ndarray'>


#### Compruebe las dimensiones de sus variables

Otra forma útil de familiarizarse con sus datos es ver sus dimensiones.

Imprima la forma de `X_train` y `y_train` y vea cuántos ejemplos de entrenamiento tiene en su conjunto de datos.

In [7]:
print ('The shape of X_train is:', X_train.shape)
print ('The shape of y_train is: ', y_train.shape)
print ('Number of training examples (m):', len(X_train))

The shape of X_train is: (10, 3)
The shape of y_train is:  (10,)
Number of training examples (m): 10


<a name="4"></a>
## 4 - Repaso del árbol de decisión

En este laboratorio de práctica, construirá un árbol de decisión basado en el conjunto de datos proporcionado.

- Recuerde que los pasos para construir un árbol de decisión son los siguientes:
    - Empezar con todos los ejemplos en el nodo raíz
    - Calcule la ganancia de información para la división en todas las características posibles y elija la que tenga la mayor ganancia de información
    - Dividir el conjunto de datos según la característica seleccionada y crear las ramas izquierda y derecha del árbol
    - Repita el proceso de división hasta que se cumplan los criterios de parada
  
  
- En este laboratorio, implementará las siguientes funciones, que le permitirán dividir un nodo en ramas izquierda y derecha utilizando la característica con mayor ganancia de información
    - Calcular la entropía en un nodo 
    - Dividir el conjunto de datos en un nodo en ramas izquierda y derecha basándose en una característica determinada
    - Calcular la ganancia de información de la división en una característica determinada
    - Elegir la característica que maximiza la ganancia de información
    
- A continuación, utilizaremos las funciones de ayuda que ha implementado para construir un árbol de decisión repitiendo el proceso de división hasta que se cumplan los criterios de parada 
    - Para este laboratorio, el criterio de parada que hemos elegido es establecer una profundidad máxima de 2

<a name="4.1"></a>
### 4.1  Calcular la entropía

Primero, escribirás una función de ayuda llamada `compute_entropy` que calcula la entropía (medida de impureza) en un nodo. 
- La función toma un array numpy (`y`) que indica si los ejemplos de ese nodo son comestibles (`1`) o venenosos (`0`) 

Completa la función `compute_entropy()` para:
* Calcular $p_1$, que es la fracción de ejemplos que son comestibles (es decir, tienen valor = `1` en `y`)
* La entropía se calcula entonces como 

$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$

* Nota 
    * El log se calcula con base $2$
    * A efectos de implementación, $0\text{log}_2(0) = 0$. Es decir, si `p_1 = 0` o `p_1 = 1`, establece la entropía a `0`.
    * Asegúrese de comprobar que los datos de un nodo no están vacíos (es decir, `len(y) != 0`). Devuelve `0` si lo está
    
<a name="ex01"></a>
### Ejercicio 1

Complete la función `compute_entropy()` siguiendo las instrucciones anteriores.
    
Si te quedas atascado, puedes consultar las pistas que se presentan después de la celda de abajo para ayudarte con la implementación.

In [8]:
# UNQ_C1
# GRADED FUNCTION: compute_entropy

def compute_entropy(y):
    """
    Calcula la entropía de 
    
    Args:
       y (ndarray): Matriz Numpy que indica si cada ejemplo en un nodo es
           comestible (`1`) o venenoso (`0`)
       
    Devuelve:
        entropía (float): La entropía en ese nodo
        
    """
    # Debe devolver correctamente las siguientes variables
    entropy = 0.
    
    ### START CODE HERE ###
    
    if len(y) != 0:
        # Su código aquí para calcular la fracción de ejemplos comestibles (es decir, con valor = 1 en y)
        p1 = len( y[ y == 1 ]) / len(y)

        # Para p1 = 0 y 1, establece la entropía en 0 (para manejar 0log0)
        if p1 != 0 and p1 != 1:
            # Su código aquí para calcular la entropía utilizando la fórmula proporcionada anteriormente
            entropy = -p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)
        else:
            entropy = 0. 
    
           
    ### END CODE HERE ###        
    
    return entropy

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
    
   * Para calcular `p1`
        * Puedes obtener el subconjunto de ejemplos en `y` que tienen el valor `1` como `y[y == 1]`
        * Puede utilizar `len(y)` para obtener el número de ejemplos en `y`.
    

   * Para calcular la "entropía"
        * <a href="https://numpy.org/doc/stable/reference/generated/numpy.log2.html">np.log2</a> Permite calcular el logaritmo en base 2 de un array de numpy
        * Si el valor de `p1` es 0 o 1, asegúrate de poner la entropía a `0`. 

    
<details>
          <summary><font size="2" color="darkblue"><b> Click for more hints</b></font></summary>
        
   * Aqui tienes la estructura para la funcion
    
```python 
    def compute_entropy(y):
        
        # You need to return the following variables correctly
        entropy = 0.

        ### START CODE HERE ###
        if len(y) != 0:
            # Your code here to calculate the fraction of edible examples (i.e with value = 1 in y)
            p1 =

            # For p1 = 0 and 1, set the entropy to 0 (to handle 0log0)
            if p1 != 0 and p1 != 1:
                # Your code here to calculate the entropy using the formula provided above
                entropy = 
            else:
                entropy = 0. 
        ### END CODE HERE ###        

        return entropy
```
    
Si sigues atascado, puedes consultar las pistas que te presentamos a continuación para saber cómo calcular `p1` y `entropía`..
    
<details>
          <summary><font size="2" color="darkblue"><b>Hint to calculate p1</b></font></summary>
           &emsp; &emsp; Puedes calcula p1 como <code>p1 = len(y[y == 1]) / len(y) </code>
    </details>

<details>
          <summary><font size="2" color="darkblue"><b>Hint to calculate entropy</b></font></summary>
          &emsp; &emsp; Puedes calcular la entropy como <code>entropy = -p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)</code>
    </details>
        
</details>

</details>

    


Puedes comprobar si tu implementación es correcta ejecutando el siguiente código de prueba:

In [9]:
# Calcula la entropía en el nodo raíz (es decir, con todos los ejemplos)
# Como tenemos 5 setas comestibles y 5 no comestibles, la entropía debería ser 1"

print("Entropy at root node: ", compute_entropy(y_train)) 

# UNIT TESTS
compute_entropy_test(compute_entropy)

Entropy at root node:  1.0
 All tests passed.


**Expected Output**:
<table>
  <tr>
    <td> <b>Entropy at root node:<b> 1.0 </td> 
  </tr>
</table>

<a name="4.2"></a>
### 4.2  Dividir el conjunto de datos

A continuación, escribirás una función de ayuda llamada `split_dataset` que toma los datos en un nodo y una característica para dividir y los divide en ramas izquierda y derecha. Más adelante en el laboratorio, implementarás el código para calcular la calidad de la división.

- La función toma los datos de entrenamiento, la lista de índices de puntos de datos en ese nodo, junto con la característica a dividir. 
- Divide los datos y devuelve el subconjunto de índices en la rama izquierda y derecha.
- Por ejemplo, digamos que empezamos en el nodo raíz (así que "índices_nodo = [0,1,2,3,4,5,6,7,8,9]`), y elegimos dividir en la característica "0", que es si el ejemplo tiene o no una gorra marrón.
    - La salida de la función es entonces, `índices_izquierda = [0,1,2,3,4,7,9]` y `índices_derecha = [5,6,8]`.
    
| Index | Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:-----:|:---------:|:--------------------:|:--------:|:------:|
|   0   |     1     |           1          |     1    |    1   |
|   1   |     1     |           0          |     1    |    1   |
|   2   |     1     |           0          |     0    |    0   |
|   3   |     1     |           0          |     0    |    0   |
|   4   |     1     |           1          |     1    |    1   |
|   5   |     0     |           1          |     1    |    0   |
|   6   |     0     |           0          |     0    |    0   |
|   7   |     1     |           0          |     1    |    1   |
|   8   |     0     |           1          |     0    |    1   |
|   9   |     1     |           0          |     0    |    0   |

<a name="ex02"></a>
### Ejercicio 2

Complete la función `split_dataset()` que se muestra a continuación

- Para cada índice en `node_indices`
    - Si el valor de `X` en ese índice para esa característica es `1`, añada el índice a `left_indices`.
    - Si el valor de `X` en ese índice para esa característica es `0`, añade el índice a `indices_derecha`.

Si te quedas atascado, puedes consultar las pistas que se presentan después de la celda de abajo para ayudarte con la implementación.

In [13]:
# UNQ_C2
# GRADED FUNCTION: split_dataset

def split_dataset(X, node_indices, feature):
    """
    Divide los datos en el nodo dado en
    ramas izquierda y derecha
    
    Args:
        X (ndarray):             Matriz de datos de forma(n_muestras, n_características)
        node_indices (lista):  Lista que contiene los índices activos. Es decir, las muestras que se están considerando en este paso.
        feature (int):           Índice de la característica a dividir en
    
    Devuelve:
        left_indices (lista): Índices con valor de característica == 1
        índices_derechos (lista): Índices con valor de característica == 0
    """
    
    # You need to return the following variables correctly
    left_indices  = []
    right_indices = []
    
    ### START CODE HERE ###
   
    # Go through the indices of examples at that node
    for i in node_indices:   
        if X[i][feature] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)

    ### END CODE HERE ###
        
    return left_indices, right_indices

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
    
   * Aqui tiene la estructura para la funcion
```python 
    def split_dataset(X, node_indices, feature):
    
        # You need to return the following variables correctly
        left_indices = []
        right_indices = []

        ### START CODE HERE ###
        # Go through the indices of examples at that node
        for i in node_indices:   
            if # Your code here to check if the value of X at that index for the feature is 1
                left_indices.append(i)
            else:
                right_indices.append(i)
        ### END CODE HERE ###
        
    return left_indices, right_indices
```
<details>
          <summary><font size="2" color="darkblue"><b> Click for more hints</b></font></summary>
        
La condicion es <code> if X[i][feature] == 1: </code>
        
</details>

</details>

    


Ahora, vamos a comprobar su implementación utilizando los bloques de código que aparecen a continuación. Intentemos dividir el conjunto de datos en el nodo raíz, que contiene todos los ejemplos en la característica 0 (Brown Cap) como habíamos comentado anteriormente

In [14]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Feel free to play around with these variables
# The dataset only has three features, so this value can be 0 (Brown Cap), 1 (Tapering Stalk Shape) or 2 (Solitary)
feature = 0

left_indices, right_indices = split_dataset(X_train, root_indices, feature)

print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

# UNIT TESTS    
split_dataset_test(split_dataset)

Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]
 All tests passed.


**Expected Output**:
```
Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]
```

<a name="4.3"></a>
### 4.3  Calcular la ganancia de información

A continuación, escribirás una función llamada `information_gain` que toma los datos de entrenamiento, los índices en un nodo y una característica para dividir y devuelve la ganancia de información de la división.

<a name="ex03"></a>
### Exercise 3

Complete la función `compute_information_gain()` que se muestra a continuación para calcular

$$\text{Information Gain} = H(p_1^\text{node})- (w^{\text{left}}H(p_1^\text{left}) + w^{\text{right}}H(p_1^\text{right}))$$

Donde
- $H(p_1^\text{node})$ is entropy at the node 
- $H(p_1^\text{left})$ and $H(p_1^\text{right})$ are the entropies at the left and the right branches resulting from the split
- $w^{\text{left}}$ and $w^{\text{right}}$ are the proportion of examples at the left and right branch, respectively

Nota:
- Puede utilizar la función `compute_entropy()` que implementó anteriormente para calcular la entropía
- Hemos proporcionado un código de inicio que utiliza la función `split_dataset()` que implementó anteriormente para dividir el conjunto de datos 

Si te quedas atascado, puedes consultar las pistas que se presentan después de la celda de abajo para ayudarte con la implementación.

In [15]:
# UNQ_C3
# GRADED FUNCTION: compute_information_gain

def compute_information_gain(X, y, node_indices, feature):
    
    """
    Calcula la información de la división del nodo en una característica dada
    
    Args:
        X (ndarray):            Matriz de datos de forma(n_muestras, n_características)
        y (array like): lista o ndarray con n_muestras que contiene la variable objetivo
        node_indices (ndarray): Lista que contiene los índices activos. Es decir, las muestras que se están considerando en este paso.
   
    Devuelve:
        coste (float):        Coste calculado
    
    """    
    # dividir Dataset
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    
    # Some useful variables
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    
    # You need to return the following variables correctly
    information_gain = 0
    
    ### START CODE HERE ###
    # Entropy at the node using compute_entropy()
    node_entropy = compute_entropy(y_node)
    # Entropy at the left branch
    left_entropy = compute_entropy(y_left)
    # Compute the entropy at the right branch
    right_entropy = compute_entropy(y_right)
    # Weights 
    w_left  = len(X_left)  / len(X_node)
    w_right = len(X_right) / len(X_node)
    #Weighted entropy
    weighted_entropy = w_left * left_entropy + w_right * right_entropy
    #Information gain                                                   
    information_gain = node_entropy - weighted_entropy
    ### END CODE HERE ###  
    
    return information_gain

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
    
   * Here's how you can structure the overall implementation for this function
```python 
    def compute_information_gain(X, y, node_indices, feature):
        # Split dataset
        left_indices, right_indices = split_dataset(X, node_indices, feature)

        # Some useful variables
        X_node, y_node = X[node_indices], y[node_indices]
        X_left, y_left = X[left_indices], y[left_indices]
        X_right, y_right = X[right_indices], y[right_indices]

        # You need to return the following variables correctly
        information_gain = 0

        ### START CODE HERE ###
        # Your code here to compute the entropy at the node using compute_entropy()
        node_entropy = 
        # Your code here to compute the entropy at the left branch
        left_entropy = 
        # Your code here to compute the entropy at the right branch
        right_entropy = 

        # Your code here to compute the proportion of examples at the left branch
        w_left = 
        
        # Your code here to compute the proportion of examples at the right branch
        w_right = 

        # Your code here to compute weighted entropy from the split using 
        # w_left, w_right, left_entropy and right_entropy
        weighted_entropy = 

        # Your code here to compute the information gain as the entropy at the node
        # minus the weighted entropy
        information_gain = 
        ### END CODE HERE ###  

        return information_gain
 ```
If you're still stuck, check out the hints below.
    
<details>
          <summary><font size="2" color="darkblue"><b> Hint to calculate the entropies</b></font></summary>
        
<code>node_entropy = compute_entropy(y_node)</code><br>
<code>left_entropy = compute_entropy(y_left)</code><br>
<code>right_entropy = compute_entropy(y_right)</code>
        
</details>
    
<details>
          <summary><font size="2" color="darkblue"><b>Hint to calculate w_left and w_right</b></font></summary>
           <code>w_left = len(X_left) / len(X_node)</code><br>
           <code>w_right = len(X_right) / len(X_node)</code>
    </details>
    
<details>
          <summary><font size="2" color="darkblue"><b>Hint to calculate weighted_entropy</b></font></summary>
           <code>weighted_entropy = w_left * left_entropy + w_right * right_entropy</code>
    </details>
    
<details>
          <summary><font size="2" color="darkblue"><b>Hint to calculate information_gain</b></font></summary>
           <code> information_gain = node_entropy - weighted_entropy</code>
    </details>


</details>


You can now check your implementation using the cell below and calculate what the information gain would be from splitting on each of the featues

In [16]:
info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)
    
info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)

# UNIT TESTS
compute_information_gain_test(compute_information_gain)

Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377
 All tests passed.


**Expected Output**:
```
Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377
```

La división en "Solitario" (característica = 2) en el nodo raíz proporciona la máxima ganancia de información. Por lo tanto, es la mejor característica para dividir en el nodo raíz.

<a name="4.4"></a>
### 4.4  Obtener la mejor división
Ahora vamos a escribir una función para obtener la mejor característica para dividir calculando la ganancia de información de cada característica como hicimos anteriormente y devolviendo la característica que da la máxima ganancia de información

<a name="ex04"></a>
### Ejercicio 4
Complete la función `get_best_split()` que se muestra a continuación.
- La función toma los datos de entrenamiento, junto con los índices de puntos de datos en ese nodo
- La salida de la función es la característica que da la máxima ganancia de información 
    - Puede utilizar la función `compute_information_gain()` para iterar a través de las características y calcular la información para cada característica
Si te quedas atascado, puedes consultar las pistas que se presentan después de la celda de abajo para ayudarte con la implementación.

In [17]:
# UNQ_C4
# GRADED FUNCTION: get_best_split

def get_best_split(X, y, node_indices):   
    """
    Devuelve la característica y el valor de umbral óptimos
    para dividir los datos del nodo 
    
    Args:
        X (ndarray):            Matriz de datos de forma(n_muestras, n_características)
        y (array like): lista o ndarray con n_muestras que contiene la variable objetivo
        node_indices (ndarray): Lista que contiene los índices activos. Es decir, las muestras que se están considerando en este paso.

    Devuelve:
        mejor_característica (int):     El índice de la mejor característica a dividir
    """    
    
    # Some useful variables
    num_features = X.shape[1]
    
    # You need to return the following variables correctly
    best_feature = -1
    
    ### START CODE HERE ###
    max_info_gain = 0

    # Iterate through all features
    for feature in range(num_features): 

       # Your code here to compute the information gain from splitting on this feature
       info_gain = compute_information_gain(X, y, node_indices, feature)

       # If the information gain is larger than the max seen so far
       if info_gain > max_info_gain:  
            max_info_gain = info_gain
            best_feature  = feature
    ### END CODE HERE ##    
   
    return best_feature

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
    
   * Here's how you can structure the overall implementation for this function
    
```python 
    def get_best_split(X, y, node_indices):   

        # Some useful variables
        num_features = X.shape[1]

        # You need to return the following variables correctly
        best_feature = -1

        ### START CODE HERE ###
        max_info_gain = 0

        # Iterate through all features
        for feature in range(num_features): 
            
            # Your code here to compute the information gain from splitting on this feature
            info_gain = 
            
            # If the information gain is larger than the max seen so far
            if info_gain > max_info_gain:  
                # Your code here to set the max_info_gain and best_feature
                max_info_gain = 
                best_feature = 
        ### END CODE HERE ##    
   
    return best_feature
```
    
If you're still stuck, check out the hints below.
    
<details>
          <summary><font size="2" color="darkblue"><b> Hint to calculate info_gain</b></font></summary>
        
    <code>info_gain = compute_information_gain(X, y, node_indices, feature)</code>
    </details>
    
<details>
          <summary><font size="2" color="darkblue"><b>Hint to update the max_info_gain and best_feature</b></font></summary>
           <code>max_info_gain = info_gain</code><br>
           <code>best_feature = feature</code>
    </details>
</details>


Now, let's check the implementation of your function using the cell below.

In [18]:
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

# UNIT TESTS
get_best_split_test(get_best_split)

Best feature to split on: 2
 All tests passed.


Como vimos anteriormente, la función devuelve que la mejor característica para dividir en el nodo raíz es la característica 2 ("Solitario")

<a name="5"></a>
## 5 - Construir el árbol

En esta sección, utilizamos las funciones que implementó anteriormente para generar un árbol de decisión eligiendo sucesivamente la mejor característica para dividir hasta que alcancemos el criterio de parada (la profundidad máxima es 2).

No es necesario implementar nada para esta parte.

In [19]:
# Not graded
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Construye un árbol utilizando el algoritmo recursivo que divide el conjunto de datos en 2 subgrupos en cada nodo.
    Esta función sólo imprime el árbol.
    
    Args:
        X (ndarray):            Matriz de datos de forma(n_muestras, n_características)
        y (array like): lista o ndarray con n_muestras que contiene la variable objetivo
        node_indices (ndarray): Lista que contiene los índices activos. Es decir, las muestras que se están considerando en este paso.
        branch_name (string):   Nombre de la rama. ['Raíz', 'Izquierda', 'Derecha']
        max_depth (int):        Profundidad máxima del árbol resultante. 
        current_depth (int):    Profundidad actual. Parámetro utilizado durante la llamada recursiva.
   
    """ 

    # Profundidad máxima alcanzada - parada de la división
    if current_depth == max_depth:
        formatting = " " * current_depth + "-" * current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices) 
    tree.append((current_depth, branch_name, best_feature, node_indices))
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)


In [20]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
